In [2]:
import torch
import pandas as pd
import numpy as np
import pickle as pkl
import scipy
import os

In [3]:
!nvidia-smi

Fri May  5 16:23:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:17:00.0 Off |                   On |
| N/A   58C    P0   185W / 300W |  32456MiB / 81920MiB |     N/A      Default |
|                               |                      |              Enabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100 80G...  On   | 00000000:65:00.0 Off |                   On |
| N/A   

In [4]:
try:
    corlat_dataset = pkl.load(open("Data/corlat/corlat.pickle", "rb"))
except:
    # move dir to /ibm/gpfs/home/yjin0055/Project/DayAheadForecast
    os.chdir("/ibm/gpfs/home/yjin0055/Project/DayAheadForecast")
    corlat_dataset = pkl.load(open("Data/corlat/corlat.pickle", "rb"))

In [4]:
# print the keys of the dataset
print("keys: ", corlat_dataset[0].keys())
print("input keys: ", corlat_dataset[0]["input"].keys())


keys:  dict_keys(['solution', 'indices', 'input'])
input keys:  dict_keys(['A', 'var_node_features', 'constraint_node_features'])


In [5]:
print("Var node features shape: ", corlat_dataset[0]["input"]["var_node_features"].shape)
print("Constraint node features shape: ", corlat_dataset[0]["input"]["constraint_node_features"].shape)

Var node features shape:  (466, 17)
Constraint node features shape:  (470, 9)


In [6]:
# names of the variable features
# 1. Variable objective coefficient
# 2. Variable type
# 3. Number of non-zero coefficients in the constraint
# 4. LP relaxation value at root node
# 5. Is LP relaxation value fractional
# 6. LP solution value equals lower bound
# 7. LP solution value equals upper bound
# 8. Has lower bound
# 9. Has upper bound
# 10. Mean degree of the constraint nodes connected to the variable
# 11. Std. deviation of the degree of the constraint nodes connected to the variable
# 12. Min. degree of the constraint nodes connected to the variable
# 13. Max. degree of the constraint nodes connected to the variable
# 14. Mean coefficient of the constraint nodes connected to the variable
# 15. Std. deviation of the coefficient of the constraint nodes connected to the variable
# 16. Min. coefficient of the constraint nodes connected to the variable
# 17. Max. coefficient of the constraint nodes connected to the variable

In [7]:
# print feature type for variable node
for i in range(corlat_dataset[0]["input"]["var_node_features"].shape[1]):
    print("feature", i, ":", type(corlat_dataset[0]["input"]["var_node_features"][0][i]))

feature 0 : <class 'numpy.str_'>
feature 1 : <class 'numpy.str_'>
feature 2 : <class 'numpy.str_'>
feature 3 : <class 'numpy.str_'>
feature 4 : <class 'numpy.str_'>
feature 5 : <class 'numpy.str_'>
feature 6 : <class 'numpy.str_'>
feature 7 : <class 'numpy.str_'>
feature 8 : <class 'numpy.str_'>
feature 9 : <class 'numpy.str_'>
feature 10 : <class 'numpy.str_'>
feature 11 : <class 'numpy.str_'>
feature 12 : <class 'numpy.str_'>
feature 13 : <class 'numpy.str_'>
feature 14 : <class 'numpy.str_'>
feature 15 : <class 'numpy.str_'>
feature 16 : <class 'numpy.str_'>


In [8]:
# print one row of the variable node features
print(corlat_dataset[0]["input"]["var_node_features"][0])

['5.0' 'B' '6' '1.0' '0.0' '0.0' '1.0' '1.0' '1.0' '19.0'
 '36.706039466732626' '1.0' '101.0' '-50.166666666666664'
 '49.83779244263891' '-100.0' '1.0']


In [9]:
# make dataframe for variable node features
for i in range(len(corlat_dataset)):
    corlat_dataset[i]["input"]["var_node_features"] = pd.DataFrame(
        corlat_dataset[i]["input"]["var_node_features"]
    )
    corlat_dataset[i]["input"]["var_node_features"].columns = [
        "var_obj_coef",
        "var_type",
        "num_nonzero_coef",
        "lp_relax_val",
        "is_lp_relax_val_frac",
        "lp_sol_val_eq_lb",
        "lp_sol_val_eq_ub",
        "has_lb",
        "has_ub",
        "mean_degree",
        "std_degree",
        "min_degree",
        "max_degree",
        "mean_coef",
        "std_coef",
        "min_coef",
        "max_coef",
    ]

In [10]:
# print head of the variable node features
corlat_dataset[0]["input"]["var_node_features"].head()

,var_obj_coef,var_type,num_nonzero_coef,lp_relax_val,is_lp_relax_val_frac,lp_sol_val_eq_lb,lp_sol_val_eq_ub,has_lb,has_ub,mean_degree,std_degree,min_degree,max_degree,mean_coef,std_coef,min_coef,max_coef
0,5.0,B,6,1.0,0.0,0.0,1.0,1.0,1.0,19.0,36.706039466732626,1.0,101.0,-50.166666666666664,49.83779244263891,-100.0,1.0
1,4.0,B,6,0.0,0.0,1.0,0.0,1.0,1.0,35.333333333333336,45.415366953879776,2.0,101.0,-48.833333333333336,51.27512934053788,-100.0,9.0
2,6.0,B,6,0.44232719895667294,1.0,0.0,0.0,1.0,1.0,35.333333333333336,45.415366953879776,2.0,101.0,-50.166666666666664,49.83779244263891,-100.0,1.0
3,5.0,B,6,-0.0,0.0,1.0,0.0,1.0,1.0,35.333333333333336,45.415366953879776,2.0,101.0,-48.833333333333336,51.27512934053788,-100.0,9.0
4,3.0,B,6,-0.0,0.0,1.0,0.0,1.0,1.0,35.333333333333336,45.415366953879776,2.0,101.0,-48.833333333333336,51.27512934053788,-100.0,9.0


In [11]:
# convert column types
# dtype for variable node features
# 1. float
# 2. str
# 3. float
# 4. float
# 5. bool
# 6. bool
# 7. bool
# 8. bool
# 9. bool
# 10. float
# 11. float
# 12. float
# 13. float
# 14. float
# 15. float
# 16. float
# 17. float
var_column_types = {
    "var_obj_coef": float,
    "var_type": str,
    "num_nonzero_coef": float,
    "lp_relax_val": float,
    "is_lp_relax_val_frac": bool,
    "lp_sol_val_eq_lb": bool,
    "lp_sol_val_eq_ub": bool,
    "has_lb": bool,
    "has_ub": bool,
    "mean_degree": float,
    "std_degree": float,
    "min_degree": float,
    "max_degree": float,
    "mean_coef": float,
    "std_coef": float,
    "min_coef": float,
    "max_coef": float,
}
for i in range(len(corlat_dataset)):
    corlat_dataset[i]["input"]["var_node_features"] = corlat_dataset[i][
        "input"
    ]["var_node_features"].astype(var_column_types)

In [12]:
# head of the variable node features
corlat_dataset[0]["input"]["var_node_features"].head()

,var_obj_coef,var_type,num_nonzero_coef,lp_relax_val,is_lp_relax_val_frac,lp_sol_val_eq_lb,lp_sol_val_eq_ub,has_lb,has_ub,mean_degree,std_degree,min_degree,max_degree,mean_coef,std_coef,min_coef,max_coef
0,5.0,B,6.0,1.000000,True,True,True,True,True,19.000000,36.706039,1.0,101.0,-50.166667,49.837792,-100.0,1.0
1,4.0,B,6.0,0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0
2,6.0,B,6.0,0.442327,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-50.166667,49.837792,-100.0,1.0
3,5.0,B,6.0,-0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0
4,3.0,B,6.0,-0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0


In [13]:
# now one hot encode var_type column
# ('C' for continuous, 'B' for binary, 'I' for integer, 'S' for semi-continuous, or 'N' for semi-integer).
# Therefore we result in a total of 5 columns for var_type after one hot encoding
# The dataset only contains 'C' and 'B' for var_type so we only need 2 columns
for i in range(len(corlat_dataset)):
    dummies = pd.get_dummies(corlat_dataset[i]["input"]["var_node_features"]["var_type"], prefix="var_type", prefix_sep="_")
    
    # simple unit test to make sure that the one hot encoding converts the var_type column to 2 columns
    assert dummies.shape[1] == 2
    
    # add the one hot encoded columns to the dataframe
    corlat_dataset[i]["input"]["var_node_features"] = pd.concat(
        [corlat_dataset[i]["input"]["var_node_features"], dummies], axis=1
    )
    
    # drop the original var_type column
    corlat_dataset[i]["input"]["var_node_features"] = corlat_dataset[i]["input"][
        "var_node_features"
    ].drop("var_type", axis=1) 

In [14]:
corlat_dataset[0]["input"]["var_node_features"].head()

,var_obj_coef,num_nonzero_coef,lp_relax_val,is_lp_relax_val_frac,lp_sol_val_eq_lb,lp_sol_val_eq_ub,has_lb,has_ub,mean_degree,std_degree,min_degree,max_degree,mean_coef,std_coef,min_coef,max_coef,var_type_B,var_type_C
0,5.0,6.0,1.000000,True,True,True,True,True,19.000000,36.706039,1.0,101.0,-50.166667,49.837792,-100.0,1.0,1,0
1,4.0,6.0,0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0,1,0
2,6.0,6.0,0.442327,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-50.166667,49.837792,-100.0,1.0,1,0
3,5.0,6.0,-0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0,1,0
4,3.0,6.0,-0.000000,True,True,True,True,True,35.333333,45.415367,2.0,101.0,-48.833333,51.275129,-100.0,9.0,1,0


In [15]:
# get feature type for constraint_node_features
for i in range(corlat_dataset[0]["input"]["constraint_node_features"].shape[1]):
    print("feature", i, ":", type(corlat_dataset[0]["input"]["constraint_node_features"][0][i]))

feature 0 : <class 'numpy.str_'>
feature 1 : <class 'numpy.str_'>
feature 2 : <class 'numpy.str_'>
feature 3 : <class 'numpy.str_'>
feature 4 : <class 'numpy.str_'>
feature 5 : <class 'numpy.str_'>
feature 6 : <class 'numpy.str_'>
feature 7 : <class 'numpy.str_'>
feature 8 : <class 'numpy.str_'>


In [16]:
# print one row of the variable constraint features
print(corlat_dataset[0]["input"]["constraint_node_features"][0])

['<' '0.0' '98' '0.7532523255945492' '5.051020408163265'
 '3.001266565899055' '-1.0' '10.0' '497.0']


In [17]:
# names of the constraint node features
# 1. Constraint type
# 2. RHS value
# 3. Number of non-zero coefficients in the constraint
# 4. Cosine similarity with the objective function
# 5. Mean of coefficients of the variables connected to the constraint
# 6. Std. deviation of coefficients of the variables connected to the constraint
# 7. Min. coefficient of the variables connected to the constraint
# 8. Max. coefficient of the variables connected to the constraint
# 9. Sum of norm of absolute values of coefficients of the variable nodes a constraint node is connected to

In [18]:
# make dataframe for constraint node features
for i in range(len(corlat_dataset)):
    corlat_dataset[i]["input"]["constraint_node_features"] = pd.DataFrame(
        corlat_dataset[i]["input"]["constraint_node_features"]
    )
    corlat_dataset[i]["input"]["constraint_node_features"].columns = [
        "constraint_type",
        "rhs",
        "num_nonzero_coef",
        "cos_sim_obj_func",
        "mean_coef",
        "std_coef",
        "min_coef",
        "max_coef",
        "sum_norm_abs_coef",
    ]

In [19]:
# convert column types
# dtype for variable node features
# 1. str
# 2. float
# 3. float
# 4. float
# 5. float
# 6. float
# 7. float
# 8. float
# 9. float
constraint_column_types = {
    "constraint_type": str,
    "rhs": float,
    "num_nonzero_coef": float,
    "cos_sim_obj_func": float,
    "mean_coef": float,
    "std_coef": float,
    "min_coef": float,
    "max_coef": float,
    "sum_norm_abs_coef": float,
}

for i in range(len(corlat_dataset)):
    corlat_dataset[i]["input"]["constraint_node_features"] = corlat_dataset[i][
        "input"
    ]["constraint_node_features"].astype(constraint_column_types)

In [20]:
# get number of unique constraint types throughout the dataset
constraint_types = set()
for i in range(len(corlat_dataset)):
    constraint_types.update(corlat_dataset[i]["input"]["constraint_node_features"]["constraint_type"].unique())

# print the unique constraint types
print(constraint_types)

{'<', '='}


In [21]:
# now one hot encode constraint_type column
for i in range(len(corlat_dataset)):
    dummies = pd.get_dummies(corlat_dataset[i]["input"]["constraint_node_features"]["constraint_type"], prefix="constraint_type", prefix_sep="_")
    
    # simple unit test to make sure that the one hot encoding converts the constraint_type column is correct shape
    assert dummies.shape[1] == 2
        
    # add the one hot encoded columns to the dataframe
    corlat_dataset[i]["input"]["constraint_node_features"] = pd.concat(
        [corlat_dataset[i]["input"]["constraint_node_features"], dummies], axis=1
    )
    
    # drop the original constraint_type column
    corlat_dataset[i]["input"]["constraint_node_features"] = corlat_dataset[i]["input"][
        "constraint_node_features"
    ].drop("constraint_type", axis=1)


In [22]:
# get head of constraint node features
corlat_dataset[0]["input"]["constraint_node_features"].head()

,rhs,num_nonzero_coef,cos_sim_obj_func,mean_coef,std_coef,min_coef,max_coef,sum_norm_abs_coef,constraint_type_<,constraint_type_=
0,0.0,98.0,0.753252,5.051020,3.001267,-1.0,10.0,497.0,1,0
1,100.0,1.0,0.000000,1.000000,0.000000,1.0,1.0,1.0,0,1
2,0.0,3.0,0.000000,0.333333,0.942809,-1.0,1.0,3.0,0,1
3,0.0,101.0,-0.872218,-0.980198,0.198020,-1.0,1.0,101.0,0,1
4,0.0,2.0,-0.082847,-49.500000,50.500000,-100.0,1.0,101.0,1,0


# Check for duplicates and drop them, for output solutions

In [23]:
def check_duplicates(arr, indices=None, drop=True):
    """
    This function takes in a list of lists and returns True if there are any duplicates, False otherwise.
    If drop=True, it also returns a new list of lists with duplicates removed.
    """
    
    # if arr is list of lists, convert to numpy array
    if isinstance(arr, list):
        arr = np.array(arr)
    
    if indices is not None:
        indexed_arr = arr[:, indices]
    
    else:
        indexed_arr = arr
    
    pairwise_comp = np.all(indexed_arr[:, np.newaxis, :] == indexed_arr[np.newaxis, :, :], axis=-1)
    duplicates = np.where(np.triu(pairwise_comp, k=1))
    if duplicates[0].size > 0:
        if drop:
            arr_unique = np.delete(arr, duplicates[0], axis=0)
            return True, arr_unique.tolist()
        else:
            return True
    else:
        if drop:
            return False, arr.tolist()
        else:
            return False

In [35]:
# for each solution, check if there are any duplicate solutions
# if there are, drop the duplicates
# first get the indices for binary variables

for i in range(len(corlat_dataset)):
    # get indices for binary variables
    binary_indices = corlat_dataset[i]["indices"]["indices"]
    
    # convert dictionary of solutions to array of arrays
    if isinstance(corlat_dataset[i]["solution"], dict):
        corlat_dataset[i]["solution"] = np.array(list(corlat_dataset[i]["solution"].values()))
    
    # check for duplicates
    has_duplicates, unique_solutions = check_duplicates(corlat_dataset[i]["solution"], indices=binary_indices)
    
    # if there are duplicates, drop them
    if has_duplicates:
        corlat_dataset[i]["solution"] = np.array(unique_solutions)[:, binary_indices]
    else:
        corlat_dataset[i]["solution"] = corlat_dataset[i]["solution"][:, binary_indices]

In [36]:
corlat_dataset[0]["solution"][:, binary_indices].shape

(1000, 100)

In [23]:
# save the dataset as corlat_preprocessed.pickle
with open("Data/corlat/corlat_preprocessed_v1.pickle", "wb") as f:
    pkl.dump(corlat_dataset, f)

In [4]:
# load the preprocessed dataset
with open("Data/corlat/corlat_preprocessed.pickle", "rb") as f:
    preprocessed_corlat_dataset = pkl.load(f)


In [5]:
preprocessed_corlat_dataset[1050]["input"]["var_node_features"].values.ravel()[::18]

array([6.0, 5.0, 7.0, 8.0, 3.0, 1.0, 4.0, 2.0, 5.0, 10.0, 2.0, 1.0, 1.0,
       1.0, 6.0, 7.0, 1.0, 3.0, 7.0, 6.0, 7.0, 3.0, 7.0, 6.0, 10.0, 6.0,
       8.0, 2.0, 9.0, 10.0, 8.0, 8.0, 9.0, 7.0, 5.0, 5.0, 2.0, 4.0, 3.0,
       4.0, 8.0, 4.0, 8.0, 7.0, 9.0, 4.0, 8.0, 6.0, 5.0, 2.0, 7.0, 1.0,
       7.0, 9.0, 5.0, 2.0, 2.0, 1.0, 2.0, 6.0, 8.0, 10.0, 4.0, 2.0, 8.0,
       9.0, 5.0, 5.0, 5.0, 1.0, 8.0, 5.0, 6.0, 9.0, 7.0, 2.0, 7.0, 3.0,
       8.0, 9.0, 3.0, 1.0, 8.0, 7.0, 1.0, 3.0, 10.0, 8.0, 8.0, 7.0, 6.0,
       3.0, 8.0, 3.0, 3.0, 4.0, 2.0, 6.0, 9.0, 3.0, 1.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [39]:
# for all solutions print shape
for i in range(len(preprocessed_corlat_dataset)):
    print(preprocessed_corlat_dataset[i]["solution"].shape)

(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(205, 100)
(1000, 100)
(451, 100)
(1000, 100)
(767, 100)
(1000, 100)
(24, 100)
(1000, 100)
(1000, 100)
(1, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(278, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(70, 100)
(1000, 100)
(1000, 100)
(93, 100)
(1000, 100)
(168, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1, 100)
(1000, 100)
(16, 100)
(249, 100)
(1000, 100)
(1000, 100)
(1, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(23, 100)
(6, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(820, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(1, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(4, 100)
(1000, 100)
(1000, 100)
(77, 100)
(1000, 100)
(1000, 100)
(1000, 100)
(5, 100)
(12, 100)
(1, 100)
(1, 100)
(1000, 100)
(15, 100)
(1000, 100)
(9, 100)
(1000, 100)
(1000, 100)
(1000, 10

In [30]:
# for all solutions print length of dictionary
for i in range(len(corlat_dataset)):
    print(len(corlat_dataset[i]["solution"]))

1000
1000
1000
1000
1000
1000
1000
205
1000
451
1000
767
1000
24
1000
1000
1
1000
1000
1000
1000
278
1000
1000
1000
1000
70
1000
1000
93
1000
168
1000
1000
1000
1000
1
1000
16
249
1000
1000
1
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
23
6
1000
1000
1000
1000
1000
1000
820
1000
1000
1000
1
1000
1000
1000
4
1000
1000
77
1000
1000
1000
5
12
1
1
1000
15
1000
9
1000
1000
1000
1000
1000
1000
1000
303
1000
1000
1000
1000
1000
2
1000
80
1000
1000
1000
1000
139
1000
1000
1000
1000
425
1000
465
2
1000
1000
1000
531
3
45
1000
1000
1000
1000
1000
1000
37
1000
1000
1000
500
1000
3
1000
1000
2
1000
1000
1000
1000
1000
42
1000
1000
1000
1000
1
183
4
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1
1000
1
1000
1000
1000
1000
1000
54
479
1000
1000
88
38
1000
1
1000
1000
1000
1000
1
7
1000
1000
1000
1000
2
1000
1000
1000
393
1000
1000
1000
1000
271
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1
1000
35
1000
1000
1000
1000
36
1000
65
6
1000
1000
1000
1000
1000
335
1000
10
1